Part I: 
- runs all loading of the .dump files
- divides the loaded data into chunks: one chunk for each snapshot
- calculates the computationally heavy calculation of disp, N_N, d_5NN etc
- saves the dataframes with all required features into the features_csv folder

NB: All clustering algorithm calculation will be done in Part II


# Clustering Algorithm applied to atom classification

- Clustering algorithm used to classify atoms as crystal-like and glass-like based on the features
- Clustering applied to each snapshot
- Program must be run from the directory that contains individual directory for each snapshot
- Each directory must be named in a pre-defined fashion (0000ps, 0100ps, 0200ps,............., 2000ps)

In [1]:
import pyprind 
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

- *.dump file contains the dump obtained from the lammps output (Has all the LAMMPS calculated quantities for each atom)
- LAMMPS does not arrange the atoms in order of their indices like DL-POLY does

In [2]:
import glob, os

def load_dump_files(file_path, label):
    '''Inputs:
        file_path=path to the directories where the dump files are located
        label = ID for the atom to be pulled out from the LAMMPS dump file
       Outputs:
        dump_contents = contents of variable values form all the snapshots in chronological order
        num_snaps = total number of snapshots present in the study       
        box = list of the box size at each snapshot
    '''
    dump_contents, box = [],[]
    files = sorted(glob.glob(file_path, recursive=True)) # sorts dump files in chronological order
    num_snaps = 0
    for filename in files:
        num_snaps += 1
        with open(filename, 'r',encoding='utf-8') as infile:
            print(filename)
            txt = infile.readlines()
            # Skip first 9 lines that contains system information not coordinates
            for line in txt[9:]:                        
                if line.split()[1] == label: 
                    dump_contents.append(np.array(line.split()).astype(float) )
            box.append(float(txt[5].split()[1]) - float(txt[5].split()[0]))
    return dump_contents, num_snaps, box


In [3]:
# label="3" for O, "2" for Nb, "1" for Li
# The description of crystallization based on the oxygen network and order is unique in the study of nucleation.
dump_contents, num_snaps, box = load_dump_files(file_path = './*/**.dump', label="3")

./0000ps/0ps.dump
./0100ps/100ps.dump
./0200ps/200ps.dump
./0300ps/300ps.dump
./0400ps/400ps.dump
./0500ps/500ps.dump
./0600ps/600ps.dump
./0700ps/700ps.dump
./0800ps/800ps.dump
./0900ps/900ps.dump
./1000ps/1000ps.dump
./1100ps/1100ps.dump
./1200ps/1200ps.dump
./1300ps/1300ps.dump
./1400ps/1400ps.dump
./1500ps/1500ps.dump
./1600ps/1600ps.dump
./1700ps/1700ps.dump
./1800ps/1800ps.dump


In [4]:
NAT=(int(len(dump_contents)/num_snaps))
print("No. of atoms: %d \nNo. of snaps: %d"%(NAT, num_snaps))


No. of atoms: 121500 
No. of snaps: 19


In [5]:
# Convert the dump_contents list into a dataframe and name the columns
df = pd.DataFrame(dump_contents)

df.columns=['ID',"Atom","x","y","z","Q4","Q6","Q8","Q10","Nc_6","Nc_8","c_x","c_y","c_z"]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2308500 entries, 0 to 2308499
Data columns (total 14 columns):
 #   Column  Dtype  
---  ------  -----  
 0   ID      float64
 1   Atom    float64
 2   x       float64
 3   y       float64
 4   z       float64
 5   Q4      float64
 6   Q6      float64
 7   Q8      float64
 8   Q10     float64
 9   Nc_6    float64
 10  Nc_8    float64
 11  c_x     float64
 12  c_y     float64
 13  c_z     float64
dtypes: float64(14)
memory usage: 246.6 MB


In [6]:
# Change the data type of the columns appropriately
df = df.astype({"ID": int, "Atom": int, "Nc_6":int, "Nc_8":int})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2308500 entries, 0 to 2308499
Data columns (total 14 columns):
 #   Column  Dtype  
---  ------  -----  
 0   ID      int64  
 1   Atom    int64  
 2   x       float64
 3   y       float64
 4   z       float64
 5   Q4      float64
 6   Q6      float64
 7   Q8      float64
 8   Q10     float64
 9   Nc_6    int64  
 10  Nc_8    int64  
 11  c_x     float64
 12  c_y     float64
 13  c_z     float64
dtypes: float64(10), int64(4)
memory usage: 246.6 MB


### Feature Engineering begins here

- Create a calculated column the find the distance from the center
- Remember the system has center at (0,0,0)
- Keep only columns that will be needed later

In [7]:
# Remember the center of our system is (0,0,0). If otherwise, you need to change the equation for df['dist_from_c']
df['dist_from_c'] = np.sqrt((df['x']**2+df['y']**2+df['z']**2))
df = df[['ID','x','y','z','Q6','Nc_6','dist_from_c']]
df.describe()

,ID,x,y,z,Q6,Nc_6,dist_from_c
count,2.308500e+06,2.308500e+06,2.308500e+06,2.308500e+06,2.308500e+06,2.308500e+06,2.308500e+06
mean,1.417505e+05,-1.698368e-02,3.999293e-02,-3.700484e-02,2.759255e-01,2.787663e+00,6.762949e+01
std,3.507404e+04,4.067301e+01,4.065108e+01,4.067496e+01,4.841351e-02,3.182917e+00,1.968527e+01
min,8.100100e+04,-7.077720e+01,-7.077650e+01,-7.077900e+01,6.639430e-02,0.000000e+00,3.410858e-01
25%,1.113758e+05,-3.517903e+01,-3.508690e+01,-3.520945e+01,2.428790e-01,1.000000e+00,5.492135e+01
50%,1.417505e+05,-6.198715e-02,6.882165e-02,-7.817165e-02,2.756860e-01,2.000000e+00,6.935329e+01
75%,1.721252e+05,3.516230e+01,3.517630e+01,3.513270e+01,3.085580e-01,3.000000e+00,8.172729e+01
max,2.025000e+05,7.077780e+01,7.077880e+01,7.077750e+01,5.171810e-01,3.300000e+01,1.216688e+02


__Looks exactly the way we want our data to be!__

- To perform clustering on each snapshot, we need to divide the dataframe into chunks for each snapshot
- Each chunk must have NAT atoms
- Each chunk needs to be sorted because the LAMMPS dump does not sort by ID

In [8]:
split_dataframes = [df[i:i+NAT] for i in range(0, len(df), NAT)]
sorted_chunks = [chunk.sort_values(by='ID') for chunk in split_dataframes]

# If you want to see what the chunks look like, uncomment the lines below
# for i, split_df in enumerate(sorted_chunks):
#     print(f"DataFrame {i + 1}:")
#     print(split_df)
#     print()

# Use the describe method to check the details of one of the chunks
# count row must have values equal to NAT
sorted_chunks[0].describe()


,ID,x,y,z,Q6,Nc_6,dist_from_c
count,121500.000000,121500.000000,121500.000000,121500.000000,121500.000000,121500.000000,121500.000000
mean,141750.500000,-0.005931,0.055560,-0.058775,0.274029,2.984041,64.324448
std,35074.173191,38.681949,38.660557,38.666512,0.045759,3.371026,18.664644
min,81001.000000,-67.020400,-67.016700,-67.021600,0.072866,0.000000,0.351533
25%,111375.750000,-33.525025,-33.378525,-33.508600,0.243665,1.000000,52.284070
50%,141750.500000,-0.048452,0.056104,-0.131143,0.275448,2.000000,65.957467
75%,172125.250000,33.465225,33.500000,33.368825,0.305249,4.000000,77.697023
max,202500.000000,67.021600,67.021900,67.021500,0.469364,30.000000,115.703800


## One more feature (Displacement)


### __How do we calculate the displacement feature?__   
- For each snapshot, calculate the disp for each atom at four different 5ps snaps just before or after the snap
- For e.g, the disp for 200ps is calculated as the average of the disp at 195 ps, 190ps, 185ps, 180ps
- disp at 195: dist(r_195ps, r_190ps)
- We need to pull out the coordinates from the HISTORY file to do so

In [9]:
def periodic_distance(atom, nbd, box):
    '''Inputs:
        atom = coordinate of a single reference atom
        nbd = array of coordinates of multiple atoms
        box = length of the box
       Outputs:
        dis = distance between atom and nbd
        len(dis) == len(nbd)
    '''
    delta = np.abs(atom-nbd)
    delta = np.where(delta> 0.5*box, delta-box, delta)
    dis = np.sqrt((delta**2).sum(axis=-1))
    return dis

In [ ]:
from tqdm import tqdm

search_string = "O2"
found = False

coords_A0=[]
with open("../SCG/HISTORY", "r") as file:
    total_lines = sum(1 for _ in file)
    file.seek(0)  # Reset the file pointer to the beginning
    progress_bar = tqdm(total=total_lines, desc="Processing")

    for line in file:
        progress_bar.update(1)  # Update the progress bar
        if search_string in line:
            found = True
        elif found:
            # Print or store the following line
            coords_A0.append(np.asarray(line.split()).astype(float))  # or do something else with the line
            found = False

    progress_bar.close()  # Close the prog

coords_A0 = np.asarray(coords_A0).astype(float)
print(np.shape(coords_A0))
print(f"Number of snaps in the HISTORY file is {int(len(coords_A0)/NAT)}")

Processing:  29%|██▊       | 41893535/145801442 [01:13<02:26, 707856.70it/s]

- 100ps.lammps was taken from the 20st set of coordinates from the A0/HISTORY file. 
- choose the neighboring sets of coordianates to calculate the displacement. 
- 100ps - 20
- 200ps - 40
- 300ps - 60
- 1300ps - 260

- if you want to pull out the 20set of coordinates you will need the slice coords_A0[19NAT:20NAT, :]

- use coords_A0 for calculating disp for 100ps, 200ps, ............., 1300ps.
 
- skipping the calculation on 0ps ( start the loop at 1, i.e. chunks for 100ps)

- choose snaps_A0 wisely: Remember how many snaps were taken from A0/HISTORY? Our last snap from A0/HISTORY was 1300ps. So our loop must run for 13 iterations (100ps, 200ps, ............, 1300ps)

In [12]:
for ind in range(1, num_snaps):
    # Locate the starting index for the snapshot 100ps (if ind=1), 200ps(if ind=2),etc.
    # 20th set will have slice indices [19*NAT, 20*NAT]
    s1 = int((ind*100)/5.-1)*NAT
    
    snap_ = coords_A0[s1: s1+NAT, :]
    nbd_1 = coords_A0[s1-NAT: s1, :]
    nbd_2 = coords_A0[s1-2*NAT: s1-NAT, :]
    nbd_3 = coords_A0[s1-3*NAT: s1-2*NAT, :]
    nbd_4 = coords_A0[s1-4*NAT: s1-3*NAT, :]

    d1 = periodic_distance(snap_, nbd_1, box[ind])
    d2 = periodic_distance(nbd_1, nbd_2, box[ind])
    d3 = periodic_distance(nbd_2, nbd_3, box[ind])
    d4 = periodic_distance(nbd_3, nbd_4, box[ind])
    print(np.mean(d1), np.mean(d2), np.mean(d3), np.mean(d4))

    dist = 0.25*(d1+d2+d3+d4)
    sorted_chunks[ind]['disp'] = dist

2.0036816824729775 2.0143528626194147 1.9931082751026103 1.9949814820022824
1.9773563895984476 1.974114796490307 1.9849747398825 1.9635903908315988
1.9782634158250947 1.9955912296172176 1.9859227171852463 1.984451497372339
1.9994675265227047 1.9687438511273974 1.9990692355308286 1.981581737301425
1.9667461780684654 1.9685253008897539 1.9558270512379028 1.9783501303317153
1.9724035064600183 1.972351733702344 1.9568223270434228 1.97436715525356
1.9747651709848288 1.9893034860453471 2.0001151621651245 1.9774670773690368
1.9328519302887153 1.9305142712394483 1.929688129031636 1.940050139492475
1.9269573440655317 1.9204502496563314 1.9399877975636641 1.9671429801930687
1.919351991596799 1.9179871330522416 1.926209322418265 1.91215046380705
1.909214728579194 1.8929509949147931 1.8913478464376348 1.9150196027623065
1.8908408206932394 1.8915065723442266 1.9046833614643277 1.8885272320362168
1.8534181902089788 1.8651392451098292 1.8726647066244104 1.8758991500062387
1.8588723767419992 1.8603122

In [13]:
sorted_chunks[13].describe()

,ID,x,y,z,Q6,Nc_6,dist_from_c,disp
count,121500.000000,121500.000000,121500.000000,121500.000000,121500.000000,121500.000000,121500.000000,121500.000000
mean,141750.500000,0.002138,0.062373,-0.014049,0.276670,2.931457,67.745669,1.866780
std,35074.173191,40.741798,40.722511,40.747027,0.048886,3.393478,19.723787,0.727076
min,81001.000000,-70.675200,-70.674700,-70.672400,0.069728,0.000000,1.173554,0.199630
25%,111375.750000,-35.271100,-35.148850,-35.222100,0.243421,1.000000,55.039559,1.379479
50%,141750.500000,-0.013427,0.101887,-0.047503,0.276251,2.000000,69.510731,1.842115
75%,172125.250000,35.231925,35.271700,35.207625,0.309573,3.000000,81.868510,2.330937
max,202500.000000,70.675700,70.674400,70.676300,0.517181,30.000000,120.169206,5.756961


__Great! We now have two new features added to our data__
- Let us do the same for the A1/HISTORY

found = False

coords_A1=[]
with open("../A1_HISTORY", "r") as file:
    total_lines = sum(1 for _ in file)
    file.seek(0)  # Reset the file pointer to the beginning
    progress_bar = tqdm(total=total_lines, desc="Processing")
    for line in file:
        progress_bar.update(1)  # Update the progress bar
        if search_string in line:
            found = True
        elif found:
            coords_A1.append(np.asarray(line.split()).astype(float))  # or do something else with the line
            found = False
    progress_bar.close()  

coords_A1 = np.asarray(coords_A1).astype(float)
print(np.shape(coords_A1))
print(f"Number of snaps in the HISTORY file is {int(len(coords_A1)/NAT)}.")

- 1400ps.lammps was taken from the 10th set of coordinates from the A1/HISTORY file. 


Let us use coords_A1 for calculating disp for 1400ps, 1500ps, ............., 2700ps.

snaps_A1 = 14
for ind in range(1,snaps_A1+1):
    s1 = int((ind*100)/5.-11)*NAT

    snap_ = coords_A1[s1: s1+NAT, :]
    nbd_1 = coords_A1[s1-NAT: s1, :]
    nbd_2 = coords_A1[s1-2*NAT: s1-NAT, :]
    nbd_3 = coords_A1[s1-3*NAT: s1-2*NAT, :]
    nbd_4 = coords_A1[s1-4*NAT: s1-3*NAT, :]
    
    d1 = periodic_distance(snap_, nbd_1, box[ind])
    d2 = periodic_distance(nbd_1, nbd_2, box[ind])
    d3 = periodic_distance(nbd_2, nbd_3, box[ind])
    d4 = periodic_distance(nbd_3, nbd_4, box[ind])
    print(np.mean(d1), np.mean(d2), np.mean(d3), np.mean(d4))


    dist = 0.25*(d1+d2+d3+d4)
    sorted_chunks[ind+snaps_A0]['disp'] = dist

sorted_chunks[26].describe()    # This is the last snapshot done

- Great the disp features has been added to 100ps, 200ps, .............., 1700ps
## More features (d_5NN, N_N)


### __How do we calculate these features?__   
- __d_5NN__: average distance of 5 nearest neighbors with Nc > 10 
- __N_N__: number of atoms with 5A that have Nc > 10

__Non parallel version of the code to calculate d_5NN and N_N__

This code is highly inefficient as you might guess because it runs only on a single processor

for ind in range(1,num_snaps):
    '''num_neigh_list = list of number of neighbors within 5A with Nc > 10
       avg_neigh_dist = average distance of 5 nearest neighbors with Nc > 10
    '''
    num_neigh_list, avg_neigh_dist = [], []
    df_snap=sorted_chunks[ind]
    print(f"Working on {ind*100} ps.")
    prog_bar = pyprind.ProgBar(len(df_snap))
    for j in range(len(df_snap)):
        filtered_particles = df_snap[df_snap['Nc_6'] > 10]
        particle_row = df_snap.iloc[j]
        coordinates = [particle_row['x'], particle_row['y'], particle_row['z']]
        distances = periodic_distance(coordinates, filtered_particles[['x','y','z']], box[ind])
        sorted_distances = np.sort(distances)
        # List of 5 nearest neighbors with Nc > 10
        # If the reference atom itself has Nc > 10, the min value of distance will be 0 and that must be removed
        nbd_5 = np.mean(sorted_distances[1:6]) if sorted_distances[0] == 0\
                else np.mean(sorted_distances[:5])
        avg_neigh_dist.append(nbd_5)
        
        num_neigh = (len([x for x in distances if x < 5.0]) -1) if min(distances) == 0\
                    else len([x for x in distances if x < 5.0])    
        num_neigh_list.append(num_neigh)
        prog_bar.update()
# Add the list as a column to the dataframe chunk
    sorted_chunks[ind]['d_5NN'] = avg_neigh_dist
    sorted_chunks[ind]['N_N'] = num_neigh_list


__d_5NN and N_N calculator using multiprocessing__

In [14]:
import multiprocessing
print("Number of CPU cores available: ",multiprocessing.cpu_count())


Number of CPU cores available:  128


In [ ]:
import multiprocessing
import pyprind

def process_snapshot(args):
    df_snap, box = args
    num_neigh_list, avg_neigh_dist = [], []
    prog_bar = pyprind.ProgBar(len(df_snap))
    for j in range(len(df_snap)):
        filtered_particles = df_snap[df_snap['Nc_6'] > 10]
        particle_row = df_snap.iloc[j]
        coordinates = [particle_row['x'], particle_row['y'], particle_row['z']]
        distances = periodic_distance(coordinates, filtered_particles[['x','y','z']], box)
        sorted_distances = np.sort(distances)
        nbd_5 = np.mean(sorted_distances[1:6]) if sorted_distances[0] == 0 else np.mean(sorted_distances[:5])
        avg_neigh_dist.append(nbd_5)
        num_neigh = (len([x for x in distances if x < 5.0]) - 1) if min(distances) == 0 else len([x for x in distances if x < 5.0])
        num_neigh_list.append(num_neigh)
        prog_bar.update()
    return num_neigh_list, avg_neigh_dist

def process_snapshots(sorted_chunks, box):
    num_snaps = len(sorted_chunks)
    # creates a multiprocessing pool object,
    # which allows you to execute functions concurrently across multiple CPUs.
    pool = multiprocessing.Pool()
    results = pool.map(process_snapshot, [(df_snap, box[ind]) for ind, df_snap in enumerate(sorted_chunks)])
    pool.close()
    pool.join()
    return results

# Example usage
# sorted_chunks is a list of dataframes containing snapshots
# box is a list of box sizes corresponding to each snapshot
results = process_snapshots(sorted_chunks, box)

# Add the results back to the dataframe chunks
for ind, (num_neigh_list, avg_neigh_dist) in enumerate(results):
    sorted_chunks[ind]['d_5NN'] = avg_neigh_dist
    sorted_chunks[ind]['N_N'] = num_neigh_list


In [ ]:
import pandas as pd
import os

# Create a directory to store CSV files
output_directory = "features_csv"
os.makedirs(output_directory, exist_ok=True)

# Loop through the DataFrames and save them as CSV files
for ind in range(1, num_snaps):
    # Define the file name (you can customize this)
    file_name = os.path.join(output_directory, f"{ind*100}.csv")
    
    # Save the DataFrame as a CSV file
    sorted_chunks[ind].to_csv(file_name, index=False)

print("CSV files saved successfully.")


__Let us look at how our variables are correlated__


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

NN_disp_corr, NN_Nc_corr, NN_dist_corr = [],[],[]
for ind in range(1,num_snaps):
    data = sorted_chunks[ind][['Q6','Nc_6','disp','N_N','dist_from_c']]
    correlation_matrix = data.corr()
    NN_dist_corr.append(correlation_matrix['N_N'].iloc[4])
    NN_disp_corr.append(correlation_matrix['N_N'].iloc[2])
    NN_Nc_corr.append(correlation_matrix['N_N'].iloc[1])
# Create a heatmap of the correlation matrix
plt.figure(figsize=(10, 8))
plt.plot(NN_disp_corr, label="NN-disp")
plt.plot(NN_dist_corr, label="NN-dist")
# plt.plot(NN_Nc_corr, label="NN-Nc")
plt.legend()
# plt.savefig("conf_mat.png",dpi=500,bbox_inches='tight')
plt.show()


In [ ]:
# Create a heatmap of the correlation matrix in the last snapshot
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.savefig("conf_mat.png",dpi=500,bbox_inches='tight')
plt.show()
